In [ ]:
import kagglehub, numpy as np, tensorflow as tf, keras
from keras.utils import audio_dataset_from_directory
from keras import layers

In [ ]:
# Download latest version
path = kagglehub.dataset_download("mohammedalkooheji/guitar-notes-dataset")

print("Path to dataset files:", path)

100%|██████████| 170M/170M [00:15<00:00, 11.2MB/s] 

Extracting files...


Path to dataset files: C:\Users\patma\.cache\kagglehub\datasets\mohammedalkooheji\guitar-notes-dataset\versions\3


In [ ]:

# training the model


trainData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "training",
    seed = 42
)


validationData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "validation",
    seed = 42
)

test_ds = audio_dataset_from_directory(
    path + "/Guitar Dataset",
    shuffle=False
)

Found 1452 files belonging to 43 classes.
Using 1162 files for training.
Found 1452 files belonging to 43 classes.
Using 290 files for validation.
Found 1452 files belonging to 43 classes.


In [ ]:
# type(trainData)
for element in trainData:
    print(element)
    break

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(88200, 1)),        # 2 seconds at 44.1 kHz
    keras.layers.Conv1D(32, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
    keras.layers.Conv1D(64, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
 
    keras.layers.Conv1D(128, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
 
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(43, activation="softmax")
])
model.summary()

data batch shape: (32, 88200, 1)
labels batch shape: (32,)


In [ ]:
#
model.compile(
    # optimizer="adam",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_2_1/conv2d_6_1/Relu defined at (most recent call last):
<stack traces unavailable>
Computed output size would be negative: -1 [input_size: 1, effective_filter_size: 3, stride: 1]
	 [[{{node StatefulPartitionedCall/functional_2_1/conv2d_6_1/Relu}}]] [Op:__inference_one_step_on_iterator_2893]

In [ ]:
# 
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

# 
history = model.fit(
    trainData,
    epochs=20,
    validation_data=validationData,
    callbacks=callbacks
)

In [ ]:
test_model = keras.models.load_model(
    "convnet_from_scratch.keras"
)
test_loss, test_acc = test_model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")